In [185]:
import plaidml.keras
plaidml.keras.install_backend()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import adfuller

from fbprophet import Prophet

import math

# import pyflux as pf

import warnings
warnings.filterwarnings('ignore')

import itertools

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import TimeSeriesSplit

from pandas.plotting import autocorrelation_plot

import re

import sys
import os

from functools import reduce

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM
#from keras.layers import Concatenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pickle

from numpy.random import seed

from tensorflow import set_random_seed

In [186]:
seed(2019)
set_random_seed(2019)

## Importing Data

In [187]:
df_combined = pd.read_pickle('./processed_data/df_combined.pickle')

In [188]:
df_combined.head(2)

,ds,y,vix,gold
0,2017-02-08 00:00:00,1054.03,18.879,2732.0
1,2017-02-08 01:00:00,1060.48,18.915,2693.0


## Functions

In [189]:
def split_train_test_chronological(df, ratio = 0.9, use_ratio = True, index = 1000):
    
    '''
    Input is a dataframe, and a ratio. Splits dataframe into 2 dataframes chronologically.
    Returns first dataframe up to the index of the length of the input dataframe times the input ratio, 
    and returns second dataframe of remaining elements.
    use_ratio is a flag, wether ratio should be used or indicies instead.
    
    df = input dataframe
    ratio = ratio to be used for splitting
    use_ratio = if True, use ratio, 
    index = index to split input dataframe on
    
    '''
    if use_ratio:
        size = len(df) * ratio
        size_round = round(size)

        df_train = df[0:(size_round)]
        df_test = df[size_round:]
    else:
        df_train = df[0:(index)]
        df_test = df[index:]
    return df_train, df_test

In [190]:
def slice_data(train_perc, stocks_to_trade, return_df_2):
    train_len = int(return_df.shape[0] * (train_perc))
    volume_to_trade = ['v_'+ticker for ticker in stocks_to_trade]
    for i in volume_to_trade:
        stocks_to_trade.append(i)

    train = return_df_2[stocks_to_trade][1:train_len].copy()
    train['diff'] = train[stocks_to_trade[0]] - train[stocks_to_trade[1]]
    train.reset_index(inplace=True, drop=True)

    test = return_df_2[stocks_to_trade][train_len:-400].copy()
    test['diff'] = test[stocks_to_trade[0]] - test[stocks_to_trade[1]]
    test.reset_index(inplace=True, drop=True)

    true_test = return_df_2[stocks_to_trade][-400:].copy()
    true_test['diff'] = true_test[stocks_to_trade[0]] - true_test[stocks_to_trade[1]]
    true_test.reset_index(inplace=True, drop=True)

    feature_names = volume_to_trade
    feature_names.append('diff')
    
    return train, test, true_test, feature_names


In [191]:
def make_variables(df, lags, look_back, predict_window):
    
    '''
    In order to use keras LSTM, we need to convert the input into a keras-friendly input.
    
    df = input dataframe
    lags = number of lags
    look_back = number of preceding elements to be considered
    predict_window = size of window for predictions
    
    '''
    
    features = lags + 1
    start = look_back
    stop = len(df) - lags - predict_window

    lstm_in_X = np.zeros(shape=(stop-start, look_back+1, features))
    lstm_in_Y = np.zeros(shape=(stop-start, look_back+1))

    iter_list = [num for num in range(look_back+1)][::-1]
    for i in range(start, stop):
        for index, j in enumerate(iter_list):
            X = df[i - j : i - j + lags + 1, -1]
            lstm_in_X[i - start, index] = np.ravel(X)
            Y = df[i - j + lags + 1, -1]
            lstm_in_Y[i-start, index] = Y
            
    return lstm_in_X, lstm_in_Y, features

In [192]:
def train_lstm(train_X, train_Y, lags, look_back, predict_window, lstm_nodes,
               dense_layers, dropout = 0.1, loss_type = 'hinge', optimizer_type = 'adam',
               number_epochs = 300, batch_size = 64, ):
    

    model = Sequential()
    model.add(LSTM(lstm_nodes, input_shape=(look_back+1, features)))
    model.add(Dropout(dropout))
    for nodes in dense_layers:
        model.add(Dense(nodes))
        model.add(Dropout(dropout))
    model.add(Dense(look_back + 1))
    model.compile(loss=loss_type, optimizer= optimizer_type)
    model.fit(train_X, train_Y, epochs=number_epochs, batch_size = batch_size, verbose = 1)
    
    pred_Y_train = model.predict(train_X)
    predictions = pred_Y_train[:,-1]
    actuals = train_Y[:,-1]
    
    print(model.summary())
    
    return model, dataset, train_X, train_Y, predictions, actuals

In [193]:
def make_refitting_window(model, train_x, train_y, test_x, test_y):
    predictions_test = []
    actuals_test = []
    
    for i in range(0, len(test_y) - 1):
        model.fit(train_x, train_y, 
                    epochs= 2, 
                    batch_size = 128, 
                    validation_data=(test_x[i].reshape(1,1,23), (test_y[i].reshape(1 ,))),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)
        pred_Y_test = model.predict(test_x)
        train_x = np.concatenate((train_x, (test_x[i].reshape(1 , 1 , 23))))
        le = len(test_y) + 1
        train_y = np.concatenate((train_y, (test_y[i].reshape(1 ,))))
        predict_test = pred_Y_test[-1,-1]
        actual_test = train_x[-1,-1]
        predictions_test.append(predict_test)
        actuals_test.append(actual_test)
        
    
    return predictions_test, actuals_test

In [194]:
#     predictions_test = []
#     actuals_test = []
    
#     for i in range(0, len(test_y) - 1):
#         model.fit(train_x, train_y, 
#                     epochs= 2, 
#                     batch_size = 128, 
# #                     validation_data=(test_x[i], test_y[i]),
#                     verbose=2,
# #                     callbacks=[earlystopper],
#                     shuffle=False)
#         pred_Y_test = model.predict(test_x)
#         test_x = np.concatenate((test_x, (test_x[i].reshape(1 , 1 , 23))))
#         le = len(test_y) + 1
#         test_y = np.concatenate((test_y, (test_y[i].reshape(1 ,))))
#         predict_test = pred_Y_test[-1,-1]
#         actual_test = train_x[-1,-1]
#         predictions_test.append(predict_test)
#         actuals_test.append(actual_test)
        
    
#     return predictions_test, actuals_test

In [195]:

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    '''
    
    
    '''
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Train, Val, Test split

In [196]:
df_train, df_test = split_train_test_chronological(df_combined, ratio = .99)

In [197]:
df_train, df_val = split_train_test_chronological(df_combined, ratio = .95)

## Modeling

In [198]:
# load dataset
values = df_combined.drop('ds', axis = 1).values
# integer encode direction
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 1)

reframed_2 = series_to_supervised(values, 7, 1)
reframed.head()

,var1(t-7),var2(t-7),var3(t-7),var1(t-6),var2(t-6),var3(t-6),var1(t-5),var2(t-5),var3(t-5),var1(t-4),...,var3(t-3),var1(t-2),var2(t-2),var3(t-2),var1(t-1),var2(t-1),var3(t-1),var1(t),var2(t),var3(t)
7,0.005247,0.179461,0.003901,0.005594,0.180275,0.001623,0.005727,0.179032,0.002740,0.005882,...,0.004491,0.005908,0.179438,0.004092,0.005895,0.179642,0.003908,0.005905,0.179642,0.003899
8,0.005594,0.180275,0.001623,0.005727,0.179032,0.002740,0.005882,0.176997,0.004602,0.006007,...,0.004092,0.005895,0.179642,0.003908,0.005905,0.179642,0.003899,0.004164,0.179642,0.002629
9,0.005727,0.179032,0.002740,0.005882,0.176997,0.004602,0.006007,0.176997,0.004491,0.005908,...,0.003908,0.005905,0.179642,0.003899,0.004164,0.179642,0.002629,0.004071,0.179642,0.003160
10,0.005882,0.176997,0.004602,0.006007,0.176997,0.004491,0.005908,0.179438,0.004092,0.005895,...,0.003899,0.004164,0.179642,0.002629,0.004071,0.179642,0.003160,0.004373,0.179642,0.003959
11,0.006007,0.176997,0.004491,0.005908,0.179438,0.004092,0.005895,0.179642,0.003908,0.005905,...,0.002629,0.004071,0.179642,0.003160,0.004373,0.179642,0.003959,0.004777,0.179642,0.003199


In [199]:
# drop columns we don't want to predict
y = reframed.iloc[:,-11].values
X = reframed.drop('var1(t)', axis =1).values
# split into train and test sets

n_train_hours = 12000
n_test_hours = 320
train_X = X[:n_train_hours,:]
train_y = y[:n_train_hours]

val_X= X[n_train_hours:-n_test_hours,]
val_y= y[n_train_hours:-n_test_hours]

test_X = X[-n_test_hours:,:]
test_y = y[-n_test_hours:]



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

(12000, 1, 23) (12000,) (341, 1, 23) (341,) (320, 1, 23) (320,)


In [202]:
# # drop columns we don't want to predict
# y = reframed_2.iloc[:,-11].values
# X = reframed_2.drop('var1(t)', axis =1).values
# # split into train and test sets

# n_train_hours = 12000
# n_test_hours = 320
# train_X = X[:n_train_hours,:]
# train_y = y[:n_train_hours]

# val_X= X[n_train_hours:-n_test_hours,]
# val_y= y[n_train_hours:-n_test_hours]

# test_X = X[-n_test_hours:,:]
# test_y = y[-n_test_hours:]



# # reshape input to be 3D [samples, timesteps, features]
# train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
# val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
# test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
# print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

In [204]:
LSTM_model_3 = Sequential()
LSTM_model_3.add(LSTM(64, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.05,recurrent_dropout=0.05))
LSTM_model_3.add(Dense(64))
# LSTM_model_3.add(LSTM(16, activation='relu'))
LSTM_model_3.add(Dense(1))
LSTM_model_3.compile(optimizer='adam', loss='mse')





In [205]:
history_3 = LSTM_model_3.fit(train_X, train_y, 
                    epochs= 100, 
                    batch_size = 128, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

Train on 12000 samples, validate on 341 samples
Epoch 1/100
 - 2s - loss: 73327.3796 - val_loss: 177.9110
Epoch 2/100
 - 2s - loss: 2701.4853 - val_loss: 481.3569
Epoch 3/100
 - 2s - loss: 1154.6047 - val_loss: 560.7679
Epoch 4/100
 - 2s - loss: 569.0086 - val_loss: 295.9989
Epoch 5/100
 - 2s - loss: 411.9404 - val_loss: 301.1240
Epoch 6/100
 - 2s - loss: 349.9914 - val_loss: 318.5336
Epoch 7/100
 - 2s - loss: 311.3779 - val_loss: 296.5194
Epoch 8/100
 - 2s - loss: 288.4732 - val_loss: 291.8084
Epoch 9/100
 - 2s - loss: 259.1932 - val_loss: 302.5205
Epoch 10/100
 - 2s - loss: 267.0308 - val_loss: 285.9072
Epoch 11/100
 - 2s - loss: 261.2335 - val_loss: 305.0317
Epoch 12/100
 - 2s - loss: 254.0558 - val_loss: 311.7035
Epoch 13/100
 - 2s - loss: 259.2388 - val_loss: 306.1729
Epoch 14/100
 - 2s - loss: 254.5865 - val_loss: 306.9204
Epoch 15/100
 - 2s - loss: 247.7920 - val_loss: 299.5172
Epoch 16/100
 - 2s - loss: 238.7192 - val_loss: 287.3755
Epoch 17/100
 - 2s - loss: 256.7868 - val_los

In [206]:
def make_refitting_window(model, train_x, train_y, test_x, test_y):
    predictions_test = []
    actuals_test = []
    
    for i in range(0, len(test_y) - 1):
        model.fit(train_x, train_y, 
                    epochs= 2, 
                    batch_size = 128, 
                    validation_data=(test_x[i].reshape(1,1,23), (test_y[i].reshape(1 ,))),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)
        pred_Y_test = model.predict(test_x)
        train_x = np.concatenate((train_x, (test_x[i].reshape(1 , 1 , 23))))
        le = len(test_y) + 1
        train_y = np.concatenate((train_y, (test_y[i].reshape(1 ,))))
        predict_test = pred_Y_test[-1,-1]
        actual_test = train_x[-1,-1]
        predictions_test.append(predict_test)
        actuals_test.append(actual_test)
        
    
    return predictions_test, actuals_test

In [207]:
res = make_refitting_window(LSTM_model_3, train_X, train_y, val_X, val_y)

Train on 12000 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.6094 - val_loss: 81.6133
Epoch 2/2
 - 2s - loss: 76.9663 - val_loss: 80.5709
Train on 12001 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.3052 - val_loss: 79.6638
Epoch 2/2
 - 1s - loss: 76.0717 - val_loss: 78.8749
Train on 12002 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 75.9548 - val_loss: 78.1809
Epoch 2/2
 - 2s - loss: 75.9781 - val_loss: 77.6605
Train on 12003 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.2601 - val_loss: 77.1252
Epoch 2/2
 - 2s - loss: 78.8641 - val_loss: 76.9604
Train on 12004 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.8638 - val_loss: 76.4889
Epoch 2/2
 - 2s - loss: 86.3460 - val_loss: 77.2550
Train on 12005 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.8443 - val_loss: 76.9335
Epoch 2/2
 - 2s - loss: 76.5043 - val_loss: 76.7730
Train on 12006 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 76.5458 - val_loss: 76.6073
Ep

 - 6s - loss: 81.7647 - val_loss: 167.6141
Epoch 2/2
 - 2s - loss: 81.7662 - val_loss: 167.6100
Train on 12055 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.7734 - val_loss: 167.6071
Epoch 2/2
 - 2s - loss: 81.7720 - val_loss: 167.6034
Train on 12056 samples, validate on 1 samples
Epoch 1/2
 - 4s - loss: 81.7894 - val_loss: 167.5806
Epoch 2/2
 - 2s - loss: 81.7821 - val_loss: 167.6070
Train on 12057 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.7830 - val_loss: 167.5961
Epoch 2/2
 - 2s - loss: 81.7829 - val_loss: 167.5912
Train on 12058 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.7876 - val_loss: 167.5899
Epoch 2/2
 - 2s - loss: 81.7854 - val_loss: 167.5871
Train on 12059 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.7912 - val_loss: 167.5852
Epoch 2/2
 - 2s - loss: 81.7889 - val_loss: 167.5825
Train on 12060 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.7945 - val_loss: 167.5807
Epoch 2/2
 - 2s - loss: 81.7921 - val_loss: 1

 - 5s - loss: 81.9025 - val_loss: 167.5313
Epoch 2/2
 - 2s - loss: 81.9011 - val_loss: 167.5566
Train on 12109 samples, validate on 1 samples
Epoch 1/2
 - 4s - loss: 81.9043 - val_loss: 167.5345
Epoch 2/2
 - 2s - loss: 81.9031 - val_loss: 167.5322
Train on 12110 samples, validate on 1 samples
Epoch 1/2
 - 4s - loss: 81.9083 - val_loss: 179.9200
Epoch 2/2
 - 2s - loss: 81.9061 - val_loss: 179.9202
Train on 12111 samples, validate on 1 samples
Epoch 1/2
 - 4s - loss: 81.9122 - val_loss: 198.0333
Epoch 2/2
 - 2s - loss: 81.9098 - val_loss: 198.0330
Train on 12112 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.9173 - val_loss: 205.1599
Epoch 2/2
 - 2s - loss: 81.9147 - val_loss: 205.1588
Train on 12113 samples, validate on 1 samples
Epoch 1/2
 - 4s - loss: 81.9228 - val_loss: 222.9747
Epoch 2/2
 - 2s - loss: 81.9202 - val_loss: 222.9734
Train on 12114 samples, validate on 1 samples
Epoch 1/2
 - 5s - loss: 81.9297 - val_loss: 227.4745
Epoch 2/2
 - 2s - loss: 81.9269 - val_loss: 2

 - 2s - loss: 82.1840 - val_loss: 173.4890
Epoch 2/2
 - 3s - loss: 82.1943 - val_loss: 173.5377
Train on 12163 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.1937 - val_loss: 175.1614
Epoch 2/2
 - 2s - loss: 82.1981 - val_loss: 175.2226
Train on 12164 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.1982 - val_loss: 165.3032
Epoch 2/2
 - 2s - loss: 82.1984 - val_loss: 165.3323
Train on 12165 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.2001 - val_loss: 156.9709
Epoch 2/2
 - 2s - loss: 82.2004 - val_loss: 156.9947
Train on 12166 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.2021 - val_loss: 157.0087
Epoch 2/2
 - 2s - loss: 82.2024 - val_loss: 157.0293
Train on 12167 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.2047 - val_loss: 157.0399
Epoch 2/2
 - 2s - loss: 82.2044 - val_loss: 157.0548
Train on 12168 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.2073 - val_loss: 157.0628
Epoch 2/2
 - 2s - loss: 82.2066 - val_loss: 1

 - 2s - loss: 82.4797 - val_loss: 208.5829
Epoch 2/2
 - 2s - loss: 82.4780 - val_loss: 208.5829
Train on 12217 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.4870 - val_loss: 208.5834
Epoch 2/2
 - 2s - loss: 82.4852 - val_loss: 208.5835
Train on 12218 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.4941 - val_loss: 208.5838
Epoch 2/2
 - 2s - loss: 82.4923 - val_loss: 208.5841
Train on 12219 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.4917 - val_loss: 208.5932
Epoch 2/2
 - 2s - loss: 83.8657 - val_loss: 209.1091
Train on 12220 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.4772 - val_loss: 208.6635
Epoch 2/2
 - 2s - loss: 82.4945 - val_loss: 208.5976
Train on 12221 samples, validate on 1 samples
Epoch 1/2
 - 3s - loss: 82.5071 - val_loss: 208.5903
Epoch 2/2
 - 2s - loss: 82.5064 - val_loss: 208.5895
Train on 12222 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.5158 - val_loss: 208.5896
Epoch 2/2
 - 2s - loss: 82.5146 - val_loss: 2

 - 2s - loss: 81.3995 - val_loss: 209.9507
Epoch 2/2
 - 2s - loss: 81.4545 - val_loss: 209.8796
Train on 12271 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 81.5200 - val_loss: 209.8099
Epoch 2/2
 - 2s - loss: 81.5745 - val_loss: 209.7422
Train on 12272 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 81.6391 - val_loss: 209.6770
Epoch 2/2
 - 2s - loss: 81.6926 - val_loss: 209.6139
Train on 12273 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 81.7559 - val_loss: 209.5534
Epoch 2/2
 - 2s - loss: 81.8079 - val_loss: 209.4954
Train on 12274 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 81.8695 - val_loss: 209.4400
Epoch 2/2
 - 2s - loss: 81.9195 - val_loss: 209.3871
Train on 12275 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 81.9789 - val_loss: 209.3369
Epoch 2/2
 - 2s - loss: 82.0266 - val_loss: 209.2892
Train on 12276 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 82.0835 - val_loss: 209.2444
Epoch 2/2
 - 2s - loss: 82.1284 - val_loss: 2

 - 2s - loss: 83.0659 - val_loss: 156.3181
Epoch 2/2
 - 2s - loss: 83.0644 - val_loss: 156.3192
Train on 12325 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0690 - val_loss: 156.3203
Epoch 2/2
 - 2s - loss: 83.0675 - val_loss: 156.3214
Train on 12326 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0721 - val_loss: 198.3271
Epoch 2/2
 - 2s - loss: 83.0710 - val_loss: 198.3278
Train on 12327 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0802 - val_loss: 185.3206
Epoch 2/2
 - 2s - loss: 83.0764 - val_loss: 185.3123
Train on 12328 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0842 - val_loss: 23.2505
Epoch 2/2
 - 2s - loss: 83.0827 - val_loss: 23.2480
Train on 12329 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0762 - val_loss: 23.2514
Epoch 2/2
 - 2s - loss: 83.0766 - val_loss: 23.2591
Train on 12330 samples, validate on 1 samples
Epoch 1/2
 - 2s - loss: 83.0693 - val_loss: 23.2635
Epoch 2/2
 - 2s - loss: 83.0698 - val_loss: 23.270

In [213]:
plt.figure(figsize=(18,8))
# sns.lineplot(data=res[1])
# sns.lineplot(data=res[0])

<Figure size 1296x576 with 0 Axes>

<Figure size 1296x576 with 0 Axes>

In [209]:
res[0]

[33.601868,
 33.696842,
 33.76548,
 33.80529,
 33.788517,
 33.81598,
 33.83765,
 33.834778,
 33.855114,
 33.797737,
 33.84174,
 33.863747,
 33.87972,
 33.89427,
 33.899647,
 33.91391,
 33.933064,
 33.949173,
 33.95564,
 33.971054,
 33.983288,
 33.98824,
 34.008656,
 34.019253,
 34.03611,
 34.043926,
 34.0554,
 34.06782,
 34.075394,
 34.07898,
 34.083244,
 34.091904,
 34.094402,
 34.179165,
 34.186157,
 34.189075,
 34.190224,
 34.192646,
 34.192097,
 34.193157,
 34.19435,
 34.193645,
 34.192234,
 34.196922,
 34.190254,
 34.193634,
 34.198116,
 34.198093,
 34.199295,
 34.199593,
 34.20014,
 34.20039,
 34.200993,
 34.20143,
 34.201572,
 34.201824,
 34.201687,
 34.202297,
 34.202454,
 34.202633,
 34.202797,
 34.19766,
 34.20315,
 34.20323,
 34.203056,
 34.19792,
 34.20352,
 34.20365,
 34.203743,
 34.20379,
 34.203903,
 34.20358,
 34.20404,
 34.204105,
 34.20414,
 34.204216,
 34.204266,
 34.20431,
 34.204353,
 34.204395,
 34.20443,
 34.20446,
 34.20449,
 34.18895,
 34.203083,
 34.203945,
 3

In [210]:
res[1]

[array([3733.65 ,   42.728, 3806.16 , 3725.8  ,   42.578, 3766.11 ,
        3734.95 ,   42.578, 3793.27 , 3699.15 ,   42.578, 3780.49 ,
        3712.18 ,   42.578, 3761.76 , 3742.8  ,   42.578, 3774.97 ,
        3758.81 ,   42.578, 3782.67 ,   42.578, 3824.77 ], dtype=float32),
 array([3725.8  ,   42.578, 3766.11 , 3734.95 ,   42.578, 3793.27 ,
        3699.15 ,   42.578, 3780.49 , 3712.18 ,   42.578, 3761.76 ,
        3742.8  ,   42.578, 3774.97 , 3758.81 ,   42.578, 3782.67 ,
        3805.71 ,   42.578, 3824.77 ,   42.578, 3863.66 ], dtype=float32),
 array([3734.95 ,   42.578, 3793.27 , 3699.15 ,   42.578, 3780.49 ,
        3712.18 ,   42.578, 3761.76 , 3742.8  ,   42.578, 3774.97 ,
        3758.81 ,   42.578, 3782.67 , 3805.71 ,   42.578, 3824.77 ,
        3813.19 ,   42.578, 3863.66 ,   42.578, 3880.15 ], dtype=float32),
 array([3699.15 ,   42.578, 3780.49 , 3712.18 ,   42.578, 3761.76 ,
        3742.8  ,   42.578, 3774.97 , 3758.81 ,   42.578, 3782.67 ,
        3805.71 ,   42.578,